# Capítulo 1: Introdução ao DuckDB e S3

Este notebook demonstra os conceitos básicos do DuckDB e sua integração com S3.

## Setup Inicial

In [ ]:
# Imports
import duckdb
import os
from pathlib import Path
import pandas as pd

# Configuração para Windows (UTF-8)
if os.name == 'nt':
    os.system('chcp 65001 > nul')

print("✓ Imports realizados com sucesso!")

## 1. O que é DuckDB?

In [ ]:
# Criar conexão com DuckDB (em memória)
conn = duckdb.connect(':memory:')

# Verificar versão
version = conn.execute("SELECT version()").fetchone()[0]
print(f"DuckDB Version: {version}")

## 2. Criando Dados de Exemplo

In [ ]:
# Criar diretório para dados
data_dir = Path("./sample_data")
data_dir.mkdir(exist_ok=True)

# Criar tabela de vendas
conn.execute("""
    CREATE TABLE sales AS
    SELECT
        range as id,
        'Product_' || (range % 10) as product,
        (random() * 1000)::INTEGER as amount,
        DATE '2024-01-01' + INTERVAL (range % 90) DAY as date
    FROM range(1000)
""")

# Visualizar dados
conn.execute("SELECT * FROM sales LIMIT 10").fetchdf()

## 3. Escrita de Arquivo Parquet

In [ ]:
# Escrever para Parquet
parquet_file = data_dir / "sales.parquet"
conn.execute(f"COPY sales TO '{parquet_file}' (FORMAT parquet)")

print(f"✓ Arquivo criado: {parquet_file}")
print(f"✓ Tamanho: {parquet_file.stat().st_size:,} bytes")

## 4. Leitura Direta de Parquet

In [ ]:
# Ler arquivo Parquet
conn.execute(f"SELECT * FROM '{parquet_file}' LIMIT 5").fetchdf()

In [ ]:
# Análise agregada
conn.execute(f"""
    SELECT
        product,
        count(*) as transactions,
        sum(amount) as total_sales,
        avg(amount)::INTEGER as avg_amount
    FROM '{parquet_file}'
    GROUP BY product
    ORDER BY total_sales DESC
""").fetchdf()

## 5. Formatos Suportados

In [ ]:
# CSV
csv_file = data_dir / "sales.csv"
conn.execute(f"""
    COPY (SELECT * FROM sales LIMIT 100)
    TO '{csv_file}' (FORMAT csv, HEADER true)
""")

# JSON
json_file = data_dir / "sales.json"
conn.execute(f"""
    COPY (SELECT * FROM sales LIMIT 100)
    TO '{json_file}' (FORMAT json)
""")

print(f"✓ CSV: {csv_file}")
print(f"✓ JSON: {json_file}")

## 6. Globbing (Múltiplos Arquivos)

In [ ]:
# Criar múltiplos arquivos
for i in range(3):
    file_path = data_dir / f"sales_part_{i}.parquet"
    conn.execute(f"""
        COPY (
            SELECT * FROM sales
            WHERE id BETWEEN {i * 300} AND {(i + 1) * 300 - 1}
        ) TO '{file_path}' (FORMAT parquet)
    """)
    print(f"✓ Criado: sales_part_{i}.parquet")

In [ ]:
# Ler com glob pattern
glob_pattern = str(data_dir / "sales_part_*.parquet").replace('\\', '/')
conn.execute(f"""
    SELECT count(*) as total_records
    FROM '{glob_pattern}'
""").fetchdf()

## 7. Metadados de Parquet

In [ ]:
# Ver schema
conn.execute(f"SELECT * FROM parquet_schema('{parquet_file}')").fetchdf()

In [ ]:
# Ver metadados
conn.execute(f"""
    SELECT
        file_name,
        row_group_id,
        num_values,
        total_compressed_size,
        total_uncompressed_size
    FROM parquet_metadata('{parquet_file}')
    LIMIT 5
""").fetchdf()

## 8. Análise de Vendas

In [ ]:
# Análise temporal
conn.execute(f"""
    SELECT
        date_trunc('week', date) as week,
        count(*) as transactions,
        sum(amount) as total_sales,
        avg(amount)::INTEGER as avg_order_value
    FROM '{parquet_file}'
    GROUP BY week
    ORDER BY week
    LIMIT 10
""").fetchdf()

## 9. Exercícios Práticos

In [ ]:
# Exercício 1: Criar dataset grande
conn.execute("""
    CREATE TABLE large_dataset AS
    SELECT
        range as id,
        'Category_' || (range % 5) as category,
        (random() * 10000)::INTEGER as value,
        current_timestamp() as created_at
    FROM range(10000)
""")

exercise_file = data_dir / "exercise_01.parquet"
conn.execute(f"COPY large_dataset TO '{exercise_file}' (FORMAT parquet)")

print(f"✓ Criado arquivo com 10.000 registros")

In [ ]:
# Exercício 2: Estatísticas
conn.execute(f"""
    SELECT
        min(value) as min_value,
        max(value) as max_value,
        avg(value)::INTEGER as avg_value,
        count(*) as total_records
    FROM '{exercise_file}'
""").fetchdf()

## Conclusão

Neste capítulo você aprendeu:
- ✅ O que é DuckDB e suas vantagens
- ✅ Criar e manipular dados localmente
- ✅ Trabalhar com formatos Parquet, CSV e JSON
- ✅ Usar globbing para múltiplos arquivos
- ✅ Explorar metadados de Parquet

**Próximo capítulo:** Instalação e configuração da extensão httpfs

In [ ]:
# Limpeza
conn.close()
print("✓ Conexão fechada!")